In [1]:
import numpy as np
import pandas as pd

import os.path as op
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

torch.manual_seed(42)

IMAGE_WIDTH = {5: 15, 20: 60, 60: 180}
IMAGE_HEIGHT = {5: 32, 20: 64, 60: 96}  

## load data

In [2]:
year_list = np.arange(1993,2001,1)

In [3]:
images = []
label_df = []
for year in year_list:
    images.append(np.memmap(op.join("./monthly_20d", f"20d_month_has_vb_[20]_ma_{year}_images.dat"), dtype=np.uint8, mode='r').reshape(
                        (-1, IMAGE_HEIGHT[20], IMAGE_WIDTH[20])))
    label_df.append(pd.read_feather(op.join("./monthly_20d", f"20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather")))
    
images = np.concatenate(images)
label_df = pd.concat(label_df)

print(images.shape)
print(label_df.shape)

(793019, 64, 60)
(793019, 8)


## build dataset

In [4]:
class MyDataset(Dataset):
    
    def __init__(self, img, label):
        self.img = torch.Tensor(img.copy())
        self.label = torch.Tensor(label)
        self.len = len(img)
  
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.img[idx], self.label[idx]

In [5]:
dataset = MyDataset(images, (label_df.Ret_20d > 0).values)

In [6]:
train_val_ratio = 0.7
train_dataset, val_dataset = random_split(dataset, [int(dataset.len*train_val_ratio), dataset.len-int(dataset.len*train_val_ratio)], \
                                           generator=torch.Generator().manual_seed(42))

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)

# the neural network

In [8]:
class Net_gelu_ln(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(5,3), stride=(3,1), dilation=(2,1), padding=(12,1)),
            nn.LayerNorm([64, 27, 60]),
            nn.GELU(),
            nn.MaxPool2d((2, 1), stride=(2, 1)),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(5,3), stride=(3,1), dilation=(2,1), padding=(12,1)),
            nn.LayerNorm([128, 10, 60]),
            nn.GELU(),
            nn.MaxPool2d((2, 1), stride=(2, 1)),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=(5,3), stride=(3,1), dilation=(2,1), padding=(12,1)),
            nn.LayerNorm([256, 7, 60]),
            nn.GELU(),
            nn.MaxPool2d((2, 1), stride=(2, 1)),
        )
        self.fc1 = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(46080, 2),
        )
        self.softmax = nn.Softmax(dim=1)
       
    def forward(self, x):
        x = x.reshape(-1,1,64,60)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.reshape(-1,46080)
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [9]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.)
    elif isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)

In [10]:
net_gelu_ln = Net_gelu_ln().cuda()
net_gelu_ln.apply(init_weights)
net_gelu_ln = nn.DataParallel(net_gelu_ln)

## train

In [11]:
def train_loop(dataloader, net, loss_fn, optimizer):
    
    size = len(dataloader.dataset)
    running_loss = 0.0
    total_loss = 0.0
    current = 0
    net.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to('cuda')
        y = y.to('cuda')
        y_pred = net(X)
        loss = loss_fn(y_pred, y.long())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        total_loss = (len(X) * running_loss + total_loss * current) / (len(X) + current)
        current += len(X)
        if batch % 50 == 49:
            print(f"batch: {batch+1} loss: {running_loss:>7f}  [{current:>5d}/{size:>5d}]")
        running_loss = 0.0
            
    return total_loss

In [12]:
def eval_loop(dataloader, net, loss_fn):
    
    size = len(dataloader.dataset)
    running_loss = 0.0
    total_loss = 0.0
    current = 0
    net.eval()
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to('cuda')
        y = y.to('cuda')
        y_pred = net(X)
        loss = loss_fn(y_pred, y.long())

        running_loss += loss.item()
        
        total_loss = (len(X) * running_loss + total_loss * current) / (len(X) + current)
        current += len(X)
        running_loss = 0.0
            
    return total_loss

In [13]:
# net = torch.load('/home/clidg/proj_2/pt/baseline_epoch_10_train_0.6865865240322523_eval_0.686580_.pt')

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net_gelu_ln.parameters(), lr=1e-5)

start_epoch = 0
min_val_loss = 1e9
last_min_ind = -1
early_stopping_epoch = 5

epochs = 100
for t in range(start_epoch, epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train_loop(train_dataloader, net_gelu_ln, loss_fn, optimizer)
    eval_loss = eval_loop(val_dataloader, net_gelu_ln, loss_fn)
    torch.save(net_gelu_ln, 'pt/'+'gelu_ln_epoch_{}_train_{:5f}_eval_{:5f}_.pt'.format(t+1, train_loss, eval_loss)) 
    if eval_loss < min_val_loss:
        last_min_ind = t
        min_val_loss = eval_loss
    elif t - last_min_ind >= early_stopping_epoch:
        break

print("Done!")

Epoch 1
-------------------------------
batch: 50 loss: 0.740813  [ 6400/555113]
batch: 100 loss: 0.763624  [12800/555113]
batch: 150 loss: 0.730428  [19200/555113]
batch: 200 loss: 0.745340  [25600/555113]
batch: 250 loss: 0.721366  [32000/555113]
batch: 300 loss: 0.733007  [38400/555113]
batch: 350 loss: 0.731935  [44800/555113]
batch: 400 loss: 0.735802  [51200/555113]
batch: 450 loss: 0.701158  [57600/555113]
batch: 500 loss: 0.734969  [64000/555113]
batch: 550 loss: 0.758405  [70400/555113]
batch: 600 loss: 0.697841  [76800/555113]
batch: 650 loss: 0.752321  [83200/555113]
batch: 700 loss: 0.716045  [89600/555113]
batch: 750 loss: 0.743313  [96000/555113]
batch: 800 loss: 0.759599  [102400/555113]
batch: 850 loss: 0.705442  [108800/555113]
batch: 900 loss: 0.680175  [115200/555113]
batch: 950 loss: 0.729599  [121600/555113]
batch: 1000 loss: 0.766683  [128000/555113]
batch: 1050 loss: 0.711922  [134400/555113]
batch: 1100 loss: 0.725565  [140800/555113]
batch: 1150 loss: 0.703207 